In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
import joblib

# Rechargez les données (assurez-vous d'avoir sauvegardé une version nettoyée, sinon refaites les étapes de renommage)
df = pd.read_csv('../data/raw/credit_card_default.csv', header=1)
df = df.rename(columns={'default payment next month': 'TARGET'})

In [2]:
X = df.drop('TARGET', axis=1)
y = df['TARGET']

In [3]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,    # 20% des données pour le test
    random_state=42,  # Pour que la séparation soit toujours la même
    stratify=y        # C'EST LE PARAMÈTRE LE PLUS IMPORTANT ICI !
)

In [4]:
# Colonnes catégorielles
categorical_features = ['SEX', 'EDUCATION', 'MARRIAGE']

# Colonnes numériques (on prend tout sauf les catégorielles et l'ID client)
numeric_features = [col for col in X.columns if col not in categorical_features + ['ID']]

# Les variables PAY_X sont ordinales, on les traitera comme numériques pour cette baseline
# On pourrait les traiter différemment dans un modèle plus avancé

In [5]:
# Pipeline pour les variables numériques
numeric_transformer = StandardScaler()

# Pipeline pour les variables catégorielles
categorical_transformer = OneHotEncoder(handle_unknown='ignore') # ignore les catégories non vues à l'entraînement

In [6]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough' # Garde les autres colonnes (ID) intactes
)

In [7]:
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(solver='saga',max_iter=2000, random_state=42))
])

In [8]:
# Le pipeline applique le prétraitement et entraîne le modèle sur X_train, y_train
model_pipeline.fit(X_train, y_train)
print("Modèle de référence entraîné !")

Modèle de référence entraîné !


C:\Users\Ahmad\loan\venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [9]:
# Évaluation sur le set de test
accuracy = model_pipeline.score(X_test, y_test)
print(f"Accuracy du modèle de référence sur le set de test : {accuracy:.4f}")

Accuracy du modèle de référence sur le set de test : 0.7785


In [10]:
# Sauvegarder le pipeline dans un fichier
joblib.dump(model_pipeline, 'baseline_logistic_regression_pipeline.pkl')
print("Pipeline sauvegardé !")

Pipeline sauvegardé !
